In [ ]:
import temporian as tp

import pandas as pd
import numpy as np

In [ ]:
# Generate a synthetic dataset
timestamps = np.arange(0, 100, 0.1)
n = len(timestamps)
noise = 0.1 * np.random.randn(n)
trend = 0.01 * timestamps
season = 0.4 * np.sin(0.5 * timestamps)

raw_data = pd.DataFrame(
    {
        "timestamp": timestamps,
        # "noise": noise,
        # "trend": trend,
        # "seasoon": season,
        "signal": noise + trend + season,
    }
)

raw_data

In [ ]:
# Convert the dataset into a Temporian dataset

source_data = tp.EventSet.from_dataframe(raw_data)

source_data

In [ ]:
# Plot the dataset
_ = source_data.plot()

In [ ]:
# Compute some signals
source = source_data.node()

signal = source["signal"]
trend = tp.simple_moving_average(signal, 50)
detrend = signal - trend
trend2 = tp.simple_moving_average(detrend, 10)
detrend2 = detrend - trend2
slope = trend2 - tp.sample(tp.lag(trend2, 5), trend2)

# TODO: Clean feature names with rename.
result = tp.glue(
    signal,
    tp.prefix("trend.", trend),
    tp.prefix("detrend.", detrend),
    tp.prefix("trend2.", trend2),
    tp.prefix("detrend2.", detrend2),
    tp.prefix("slope.", slope),
)

result_data = tp.evaluate(result, {source: source_data})

result_data

In [ ]:
# Plot the results
_ = result_data.plot()